#### 입력 : ./make_file/division/second_round/서울시_도로_{숫자}.csv
#### 출력 : ./make_file/division/third_round/보정_서울시_도로_{숫자}.csv 

In [283]:
import pandas as pd 
import requests
from tqdm import tqdm 
import os
import time

In [284]:
# 파일이 저장된 폴더 경로
folder_path = "./make_file/division/second_round/" 
file_encoding = "UTF-8"

output_file_url = "./make_file/division/third_round/보정_서울시_도로"

In [285]:
# 조건을 만족하는 파일 목록
valid_files = [] 

# api 사용량을 체크하는 변수
gukto_bu_api_count = 0 # 국토부 api
haengjeong_anjeonbu_api_count = 0 # 행안부 api

for temp_file_name in os.listdir(folder_path):
    file_name = temp_file_name.replace("_", " ")
    # file_name = temp_file_name
    if ("서울시 도로 " in file_name and file_name.endswith(".csv")) or ("서울시 도로 " in file_name and file_name.endswith(".csv")):
        file_path = os.path.join(folder_path, temp_file_name)  # 안전한 경로 조합

        try:
            # CSV 파일 전체 읽기
            df = pd.read_csv(file_path, encoding=file_encoding, low_memory=False)

            # 컬럼명 앞뒤 공백 제거
            df.columns = df.columns.str.strip()

            # 필요한 컬럼이 존재하는지 확인
            if {"중앙점_위도", "중앙점_경도", "행정동"}.issubset(df.columns):
                check = file_name.replace("서울시 도로 " , "") 
                temp_check_file_url = output_file_url + "_" + check
                
                output_check = os.path.isfile(temp_check_file_url) 
                
                if output_check is False :
                    valid_files.append(file_path)
                else :
                    print(f"{temp_check_file_url} 파일이 이미 있습니다.")
        except Exception as e:
            print(f"파일 {file_name} 처리 중 오류 발생: {e}")
    else :
        print(file_name)
        

.DS Store


In [286]:
len(valid_files) 

16

### 국토부 api

In [287]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
print(open_api_key_file['사이트'].value_counts())

get_gov_word_api_key = open_api_key_file[open_api_key_file['사이트'] == '국토부v-world'].values[0][1]  

사이트
공공데이터                 1
kakao_api_key_REST    1
카카오 JS                1
국토부v-world            1
GOOGLE_MAP            1
행정안전부                 1
Name: count, dtype: int64


In [288]:
# 위경도를 입력받아 행정동 정보를 반환하는 함수
def gukto_bu_dong_from_coords(latitude, longitude):
    global gukto_bu_api_count
    
    api_key = get_gov_word_api_key
    
    # 'point' 파라미터로 위경도를 하나의 문자열로 결합 (경도,위도 순으로)
    point = f"{longitude},{latitude}"
    
    # API URL에 'point' 파라미터 추가
    url = f"https://api.vworld.kr/req/address?service=address&version=2.0&request=GetAddress&point={point}&crs=EPSG:4326&type=ROAD&key={api_key}"
    
    # API 호출
    response = requests.get(url)
    gukto_bu_api_count += 1
    
    # 요청이 성공했는지 확인
    if response.status_code == 200:
        data = response.json()
        # print(data)
        try:
            # 행정동 정보 추출 (응답 형식에 따라 수정 필요)
            address = data['response']['result'][0]['structure']['level3']
            # print(address)
            return address
        except (KeyError, IndexError):
            return None 
    else:
        return None

### 행안부 api

In [289]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
open_api_key_file['사이트'].value_counts()

get_gov_api_key = open_api_key_file[open_api_key_file['사이트'] == '행정안전부'].values[0][1]  

In [290]:
# 변환할 지번 주소 입력 

def haengjeong_anjeon_bu_dong_from_address(old_address): 
    global haengjeong_anjeonbu_api_count
    url = f"https://www.juso.go.kr/addrlink/addrLinkApi.do?confmKey={get_gov_api_key}&keyword={old_address}&resultType=json"
    
    # API 요청
    response = requests.get(url)
    haengjeong_anjeonbu_api_count += 1
    
    if response.status_code == 200:
        data = response.json()
        try :
            dong_names = {item['emdNm'] for item in data['results']['juso']}
            return next(iter(dong_names), None)  # 첫 번째 요소 반환, 없으면 빈 문자열
        except :
            return None
    return None

In [ ]:
for file_path in valid_files:  
    data = pd.read_csv(file_path, encoding=file_encoding, low_memory=False)
    
    file_path = file_path.replace("./make_file/division/second_round/", "")
    file_path = file_path.replace(".csv", "")
    print(f"[ {file_path} ] NULL값 : {data['행정동'].isnull().sum()}")  

    for i in tqdm(range(len(data))): 
        if pd.isna(data['행정동'][i]):
            temp_address = data['도로명'][i]
        
            # 도로명이 리스트인 경우 첫 번째 값만 추출
            if isinstance(temp_address, list):
                temp_address = temp_address[0]  # 리스트일 경우 첫 번째 값만 가져옴
            elif isinstance(temp_address, str): 
                pass
            else:
                # 리스트도 아니고 문자열도 아닌 경우
                temp_address = str(temp_address)  # 해당 값을 문자열로 처리
        
            # 앞뒤 공백 제거 후 띄어쓰기를 기준으로 앞부분만 저장
            data.loc[i, '도로명'] = temp_address.strip().split()[0] 

            data.loc[i, '도로명'] = data.loc[i, '도로명'].replace("[\'", "")
            data.loc[i, '도로명'] = data.loc[i, '도로명'].replace("['", "")
            
            data.loc[i, '도로명'] = data.loc[i, '도로명'].replace("\'", "")
            data.loc[i, '도로명'] = data.loc[i, '도로명'].replace("'", "")
            
            data.loc[i, '도로명'] = data.loc[i, '도로명'].replace(",", "")
            data.loc[i, '도로명'] = data.loc[i, '도로명'].replace(",", "") 
            
            
            if data.loc[i, '도로명'] =="['천중로27길\'," : 
                data.loc[i, '도로명'] = "천중로27길"
            elif "[\'" in data.loc[i, '도로명'] : 
                print("dd")
                
            search_lat = data['중앙점_위도'][i]
            search_lot = data['중앙점_경도'][i]
            result_gukto_bu = gukto_bu_dong_from_coords(search_lat, search_lot)

            if result_gukto_bu is not None :
                data.loc[i, '행정동'] = result_gukto_bu 
                
            elif result_gukto_bu is None :
                search_address = data["도로명"][i]
                
                result = haengjeong_anjeon_bu_dong_from_address(search_address) 
                
                if result is None:
                    print(f"err : {search_address}")
                else: 
                    data.loc[i, '행정동'] = result 
                

    # 보정을 했는데도 null이면 삭제
    data = data.dropna(subset=['행정동'])

    print(f"데이터 길이 : {len(data)}", end="\n\n")
    try :
        temp_out_url = file_path.replace("./make_file/division/second_round/서울시_도로_" , "")  
        save_file_url = output_file_url + "_" + temp_out_url 
        # data.to_csv(save_file_url, index=False)
        print(f"{save_file_url} 파일 저장 완료.", end ="\n\n")
    except OSError as e :
        print(e)

[ 서울시_도로_11 ] NULL값 : 6854


 31%|███████████                        | 3147/10000 [00:00<00:00, 14104.20it/s]

In [ ]:
print(f"행안부 api 사용량 : {haengjeong_anjeonbu_api_count}")
print(f"국토부 api 사용량 : {gukto_bu_api_count}")